# Scratchwork: Cayley-Dickson Algebras

The purpose of this notebook is to develop the functions required to perform the Cayley-Dickson construction within the ``finite_algebras`` module.

In [20]:
import finite_algebras as alg
import numpy as np
# import re
# from cayley_dickson_utilities import *

In the Python module, ``finite_algebras``, all elements are strings.

When direct products or Cayley-Dickson algebras are constructed, the component algebras' elements are concatentated (joined) using a string delimiter (':' by default).

So, operations like scalar multiplication, negation, conjugation, etc, on the newly constructed "compound" elements, require some string manipulation to accomplish.

According to [Shafer, 1966]

$a,b,c,d,\mu \in F$, $\mu \neq 0$, then $(a, b) \times (c, d) = (a c + \mu d \overline{b}, \overline{a} d + c b)$,

where $\overline{a} = a$ and $\overline{(a, b)} = (\overline{a}, -b)$

## Test Algebras

In [2]:
F3 = alg.generate_algebra_mod_n(3, elem_name='')
F3sqr = F3.sqr()
F3quad = F3sqr.sqr()

F4 = alg.generate_algebra_mod_n(4, elem_name='')
F4sqr = F4.sqr()

F5 = alg.generate_algebra_mod_n(5, elem_name='')
F5sqr = F5.sqr()

F7 = alg.generate_algebra_mod_n(7, elem_name='')
F7sqr = F7.sqr()

In [3]:
F3cda = F3.make_cayley_dickson_algebra()
F3cda.about()


** Field **
Name: F3_CDA
Instance ID: 4612165264
Description: Cayley-Dickson algebra based on F3
Order: 9
Identity: '0:0'
Commutative? Yes
Cyclic?: Yes
  Generators: ['0:1', '0:2', '1:1', '1:2', '2:1', '2:2']
Elements:
   Index   Name   Inverse  Order
      0   '0:0'   '0:0'       1
      1   '0:1'   '0:2'       3
      2   '0:2'   '0:1'       3
      3   '1:0'   '2:0'       3
      4   '1:1'   '2:2'       3
      5   '1:2'   '2:1'       3
      6   '2:0'   '1:0'       3
      7   '2:1'   '1:2'       3
      8   '2:2'   '1:1'       3
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 7, 8, 6],
 [2, 0, 1, 5, 3, 4, 8, 6, 7],
 [3, 4, 5, 6, 7, 8, 0, 1, 2],
 [4, 5, 3, 7, 8, 6, 1, 2, 0],
 [5, 3, 4, 8, 6, 7, 2, 0, 1],
 [6, 7, 8, 0, 1, 2, 3, 4, 5],
 [7, 8, 6, 1, 2, 0, 4, 5, 3],
 [8, 6, 7, 2, 0, 1, 5, 3, 4]]
Mult. Identity: '1:0'
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 6, 3, 1, 7

In [4]:
F3sqr == F3cda

True

In [22]:
F3cda.order

9

In [21]:
F3cda2 = F3cda.make_cayley_dickson_algebra()
F3cda2.about()


** Ring **
Name: F3_CDA_CDA
Instance ID: 4686569232
Description: Cayley-Dickson algebra based on F3_CDA
Order: 81
Identity: '0:0:0:0'
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0 '0:0:0:0' '0:0:0:0'       1
      1 '0:0:0:1' '0:0:0:2'       3
      2 '0:0:0:2' '0:0:0:1'       3
      3 '0:0:1:0' '0:0:2:0'       3
      4 '0:0:1:1' '0:0:2:2'       3
      5 '0:0:1:2' '0:0:2:1'       3
      6 '0:0:2:0' '0:0:1:0'       3
      7 '0:0:2:1' '0:0:1:2'       3
      8 '0:0:2:2' '0:0:1:1'       3
      9 '0:1:0:0' '0:2:0:0'       3
     10 '0:1:0:1' '0:2:0:2'       3
     11 '0:1:0:2' '0:2:0:1'       3
     12 '0:1:1:0' '0:2:2:0'       3
     13 '0:1:1:1' '0:2:2:2'       3
     14 '0:1:1:2' '0:2:2:1'       3
     15 '0:1:2:0' '0:2:1:0'       3
     16 '0:1:2:1' '0:2:1:2'       3
     17 '0:1:2:2' '0:2:1:1'       3
     18 '0:2:0:0' '0:1:0:0'       3
     19 '0:2:0:1' '0:1:0:2'       3
     20 '0:2:0:2' '0:1:0:1'       3
     21 '0:2:1:0' '0:1:2:0'       3
 

In [23]:
F3quad == F3cda2

False

In [25]:
F3quad.zero_divisors()

['2:1:1:1',
 '2:1:2:2',
 '0:1:1:0',
 '2:2:1:2',
 '0:1:2:0',
 '2:2:2:1',
 '0:2:1:0',
 '0:2:2:0',
 '1:0:0:1',
 '1:0:0:2',
 '1:1:1:2',
 '1:1:2:1',
 '1:2:1:1',
 '1:2:2:2',
 '2:0:0:1',
 '2:0:0:2']

In [26]:
F3cda2.zero_divisors()

['0:1:1:1',
 '0:1:1:2',
 '0:1:2:1',
 '0:1:2:2',
 '0:2:1:1',
 '0:2:1:2',
 '0:2:2:1',
 '0:2:2:2',
 '1:0:1:1',
 '1:0:1:2',
 '1:0:2:1',
 '1:0:2:2',
 '1:1:0:1',
 '1:1:0:2',
 '1:1:1:0',
 '1:1:2:0',
 '1:2:0:1',
 '1:2:0:2',
 '1:2:1:0',
 '1:2:2:0',
 '2:0:1:1',
 '2:0:1:2',
 '2:0:2:1',
 '2:0:2:2',
 '2:1:0:1',
 '2:1:0:2',
 '2:1:1:0',
 '2:1:2:0',
 '2:2:0:1',
 '2:2:0:2',
 '2:2:1:0',
 '2:2:2:0']

## Code Tests

In [6]:
F3.inv('2')

'1'

In [27]:
for el in F3cda.elements:
    print(f"conj({el}) = {F3cda.conj(el)}")

conj(0:0) = 0:0
conj(0:1) = 0:2
conj(0:2) = 0:1
conj(1:0) = 1:0
conj(1:1) = 1:2
conj(1:2) = 1:1
conj(2:0) = 2:0
conj(2:1) = 2:2
conj(2:2) = 2:1


In [28]:
F3cda.split_element('1:2:3:4:5:6')

('1:2:3', '4:5:6')